<a href="https://colab.research.google.com/github/Harshitprajapati10/Makemore-pytorch/blob/main/MakeMore02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLP NN